_Softwarepraktikum Blockkurs März 2021 (WS2020/21)_ 

# NumPy Anwendungen[<sup>1</sup>](#fn1)

[Jörn Behrens](https://www.math.uni-hamburg.de/numgeo/) (joern.behrens@uni-hamburg.de)

## 1. Einführung und Ziel

Ziel dieses Notebooks ist es, erweiterte Anwendungsmöglichkeiten anhand von ausgewählten Algorithmen zu demonstrieren. Wir werden im Folgenden vier wichtige Anwendungsfelder numerischer Programmierung anhand von Beispielen beleuchten.

* Polynomielle Interpolation
* Lösung von Gleichungssystemen
* Lösung von Ausgleichsproblemen
* Integration bzw. Lösung von gewöhnlichen Differentialgleichungen

## 2. Interpolation

Wir wollen die Interpolationsroutinen in SciPy verwenden um ein besonders schwierig zu interpolierendes Problem zu lösen. Das Interpolationsproblem allgemein ist dabei gegeben:

> **Interpolationsproblem:** Sei eine Menge an Paaren $\{(t_i,w_i): i=0,\ldots,m\}$ gegeben, wobei die $t_i$  Koordinaten darstellen und $w_i$ Werte. Dann besteht das Interpolationsproblem darin, ein Polynom $p$ vom Grade $m$ zu finden, so dass $$p(t_i) = w_i.$$

Mit hilfe eines so gefundenen Polynoms, können dann Werte in den Zwischenräumen zwischen $t_i$ und $t_{i+1}$ ausgefüllt werden.

Als Beispiel wollen wir ein Interpolationsproblem von Runge lösen, das folgende Funktion, gegeben an einzelnen Punkten, interpolieren soll:
$$
\lambda(x) = \frac{1}{1+25 x^2}.
$$

Wir werden diese Funktion im Intervall $[-1,1]$ an 11 Punkten vorgeben und dann mittels linearer Interpolation und mittels (stückweise) kubischer Spline Interpolation interpolieren. Außerdem können wir noch mit Lagrange Interpolations-Polynomen versuchen ein gute Approximation zu erreichen. Leider ist dieses Problem mit äquidistanten Punkten sehr schlecht für polynomielle Interpolation geeignet.

Definieren wir also zunächst zwei Funktionen, eine um $\lambda(x)$ zu berechnen, eine zweite für die Lagrange Interpolation:

In [ ]:
def lmbda(x):
    y= 1./(1.+25*x**2)
    return y

In [ ]:
def Lagrange(x,y,xx):
    ''' Lagrange computes the Lagrange polynomial
        x: nodes
        y: interpolation values
        xx: scalar point at which to interpolate 
    '''
    from numpy import shape
    
    # determine length of data arrays
    [n,]= shape(x)
    [nn,]= shape(y)
    if n != nn:
        print('ERROR: x and y value vectors need to match in size')
        return
    
    # initialize s
    s= 0
    
    # loop over nodes
    for i in range(n):
        prod=y[i]
        for j in range(n):
            if i != j:
                prod = prod*(xx-x[j])/(x[i]-x[j])
        s= s+prod
    return s

Nun berechnen wir einerseits die Stützstellen $(t_i,w_i)$ und andererseits basierend auf einem sehr viel dichteren Gitter $x$ die Interpolierenden

In [ ]:
from numpy import linspace, zeros, shape
from scipy.interpolate import interp1d

t = linspace(-1,1,11,endpoint=True) # die Stützstellen
w = lmbda(t)                        # die Werte

lin_int = interp1d(t,w)             # Stückweise lineare Interpolation
cub_int = interp1d(t,w,kind='cubic')# Stückweise kubische Interpolation

x = linspace(-1,1,101,endpoint=True)# Feines Gitter für die Interpolierten Werte
l = lin_int(x)
c = cub_int(x)

p = zeros(shape(x))
for i in range(x.size):
    p[i]= Lagrange(t,w,x[i])        # Lagrange Interpolation

e = lmbda(x)                        # exakte Werte auf feinem Gitter

Mit Hilfe der erlernten Techniken zur Visualisierung stellen wir uns nun diese verschiedenen Interpolationen graphisch dar.

In [ ]:
import matplotlib.pyplot as plt

 # plot
h1=plt.plot(t,w,'ro',label='Daten')
h2=plt.plot(x,p,'b:',label='Lagrange')
h3=plt.plot(x,e,'g--',label='Exakt')
h4=plt.plot(x,l,'m-',label='Linear')
h5=plt.plot(x,c,'c.-',label='Kubisch')
plt.legend(loc='center right',bbox_to_anchor=(1.3, 0.5))
plt.title('Interpolation')
plt.xlabel('x/t (Stützstellen)')
plt.ylabel('y/w (Werte)');

## 3. Lösung linearer Gleichungssysteme

Eine der Kernaufgaben der numerischen linearen Algebra ist die Lösung von linearen Gleichungssystemen. Diese treten überall auf und sind - gerade wenn sie groß werden - nur mit großem Aufwand lösbar. SciPy erlaubt es einem, bei moderaten Problemgrößen mit sehr wenig Programmieraufwand solche Systeme zu lösen. Der grundlegende Befehl dafür lautet

* `solve`

und ist im SciPy Untermodul linalg enthalten. Darüber hinaus gibt es eine große Menge weiterer Löser für speziell strukturierte Matrizen; Beispiele sind:

* `solve_banded` für Bandmatrizen
* `solve_toeplitz` für Toeplitz-Matrizen
* `solve_circulant` für zyklische Matrizen

Wir werden nur eine kleine Demonstration geben. Dabei ist die Matrix recht beliebig gewählt.

In [ ]:
from scipy.linalg import solve, lstsq
from numpy import arange, ones, eye, diag, shape, loadtxt, array, transpose, multiply
from numpy.random import rand
A = 4*eye(5,5)-diag(ones(4),1)-diag(ones(4),-1)
b = rand(5)     # rechte Seite b
x = solve(A,b)  # Lösung
print('Matrix A:\n',A)
print('Rechte Seite b:\n',b)
print('Lösung:\n',x)

## 4. Lineare Ausgleichsprobleme

Lineare Ausgleichsprobleme sind sozusagen Geschwister der Interpolationsprobleme. Auch hier ist eine Menge an Paaren $\{(t_i,w_i): i=0,\ldots,m\}$ gegeben. Jedoch soll nun die gesuchte Funktion die gegebenen Werte nicht exakt erreichen, sondern im quadratischen Mittel eine beste Annäherung erreichen. Das Problem lässt sich also formulieren als

> **Ausgleichsproblem:** Sei eine Menge an Paaren $\{(t_i,w_i): i=0,\ldots,m\}$ gegeben, wobei die $t_i$  Koordinaten darstellen und $w_i$ Werte. Sei weiter $\varphi(t;x_1,\ldots,x_n)$ eine Funktion in $t$, die von Parametern $x_1,\ldots,x_n$ abhängt. Dann suchen wir diese Parameter $x_1,\ldots,x_n$, so dass $$\|\varphi(t_i;x_1,\ldots,x_n) - w_i\|^2 = \min !$$

Wir versuchen also die Differenzen zwischen der Funktion $\varphi$ und den gegebenen Werten zu minimieren. In der Regel wird dabei die Anzahl der Parameter klein gegenüber der Anzahl der Daten sein ($n\ll m$).

Wenn wir nun annehmen, dass $\varphi$ durch ein lineares Modell gegeben ist, also
$$
\varphi(t;x_1,\ldots,x_n) = a_1(t)x_1 + \cdots + a_n(t)x_n,
$$
Dann können wir das Problem als überbestimmtes System auffassen, das wir lediglich minimieren, aber nicht exakt lösen können:
$$
\min = \|w - Ax\|^2,
$$
mit
$$
w = \left[\begin{array}{c}w_0\\ \vdots \\ w_m\end{array}\right],\ 
x = \left[\begin{array}{c}x_1\\ \vdots \\ x_n\end{array}\right],\ 
A = \left[\begin{array}{ccc}
a_1(t_1) & \cdots & a_n(t_1) \\ 
\vdots & \  & \vdots \\ 
a_1(t_m) & \cdots & a_n(t_m) \\ 
\end{array}\right]
$$

### Beispiel: Klimadaten für Hamburg

Wir wollen nun ein Beispiel zeigen, in dem wir eine Datenreihe von mittleren Jahrestemperaturen für Hamburg für die Jahre 1892 bis 2010 mit einer linearen oder quadratischen Funktion annähern wollen. Diese Daten laden wir mit `loadtxt` aus einer Datei, die aus [1] herunter geladen wurde.

[1] _Hamburger Bildungsserver, Klimadaten des Deutschen Wetterdienstes (DWD) von Stationen in Norddeutschland, http://bildungsserver.hamburg.de/contentblob/2872124/data/hamburg.zip, letzter Zugriff 09.12.2019._

In [ ]:
raw=loadtxt('HHyearmean1892_2010.dat')
year=array(raw[:,0])
temp=array(raw[:,1])
[m,n]=shape(raw)

Jetzt müssen wir die Matrizen zu den beiden Funktionen erstellen. Wir wollen die lineare und quadratische Funktion
$$
\phi(x)=a+b\cdot x,\quad \psi(x)=a+b\cdot x+c\cdot x^2
$$
zur Approximation verwenden, gesucht sind also die Koeffizienten $a, b, c$. Dabei sind Werte zu den Jahren gegeben, wir stellen also die Rechte Seite aus den Temperaturwerten und die Koeffizienten der Systemmatrix $A$ aus den Jahreszahlen (d.h. den Datenpunkten) auf.

In [ ]:
Al=transpose(array([year, ones(m)]))
Aq=transpose(array([year*year, year, ones(m)]))
b=temp

Damit müssen wir nun nur noch den SciPy Befehl `lstsq` aufrufen und können damit schon unser Problem lösen!

In [ ]:
xl, resl, rnkl, sl = lstsq(Al, b)
xq, resq, rnkq, sq = lstsq(Aq, b)

Damit  haben wir die Parameter für unsere beiden Funktionen $\phi$ und $\psi$ berechnet und können diese Rekonstruieren:

In [ ]:
plinear= xl[0]*year +xl[1]
pquadr = xq[0]*multiply(year,year)+ xq[1]*year+ xq[2] 

Zu guter Letzt stellen wir das Ergebnis wieder graphisch dar. In beiden Fällen lässt sich aus den Daten ein Erwärmungstrend ablesen.

In [ ]:
plt.plot(year,temp,'ro')
plt.plot(year,plinear,'b-',linewidth=2,label='linear')
plt.plot(year,pquadr,'k-',linewidth=2,label='quadr.')
plt.legend(loc='upper left')
plt.title('Jahresmittel in Hamburg (HHyearmean1892_2010.dat)')
plt.xlabel('Jahr')
plt.ylabel('Temperatur [°C]');

## 5. Lösung gewöhnlicher Differentialgleichungen

Als Abschluss dieses Notebooks wollen wir eine komplizierte gewöhnliche Differentialgleichung lösen. Es handelt sich um ein Problem aus der Raumfahrt, das sogenannte _Arenstorf Orbit_. Ein Raumschiff soll von der Erde aus zum Mond und wieder zurück fliegen und dabei möglichst die Beschleunigungskräfte des Planeten und seines Trabanten nutzen. Das Erde-Mond System ist dabei in einem rotierenden baryzentrischen Koordinatensystem $(x1,x2)$ gegeben, so dass darin Erde und Mond immer fest stehen. Die Beschleunigung des Raumschiffes ist durch ein System von zwei gewöhnlichen Differentialgleichungen gegeben:
\begin{align}
x_1^{''} & =  x_1 + 2 x_2^\prime - \hat{\mu} \frac{x_1+\mu}{N_1}
                 - \mu \frac{x_1-\hat{\mu}}{N_2}, \\
  x_2^{''} & =  x_2 - 2 x_1^\prime - \hat{\mu} \frac{x_2}{N_1} - 
                 \mu \frac{x_2}{N_2}
\end{align}
mit $N_1 = \left( (x_1 + \mu)^2 + x_2^2\right)^{3/2}$, $N_2 = \left( (x_1-\hat{\mu})^2 + x_2^2\right)^{3/2}$ und Daten $\mu = 0.012277471$, $\hat{\mu} = 1-\mu$. $\mu$ ist dabei das Verhältnis der Masse des Mondes zur Gesamtmasse des Systems. Die Längeneinheit is auf die Distanz zwischen Mond und Erde normiert und die Zeiteinheit ist ein Monat. Anfangswerte für das System sind gegeben durch
$$
  x_1(0) = 0.994, \quad x^\prime_1(0) = 0, \quad
  x_2(0) = 0, \quad x_2^\prime(0) = -2.001585106
$$
und die Periode für einen Umlauf ist $T = 17.0652166$ Zeiteinheiten.

Um dieses Problem mit Hilfe von Python zu lösen, müssen wir zunächst ein System erster Ordnung daraus herleiten. Das gelingt mittels Hilfsvariablen $x_3:=x_1'$ und $x_4:=x_2'$. Damit können wir die Terme zweiter Ordnung in der obigen Gleichung ersetzen durch $x_1^{''} = x_3'$ und $x_2^{''} = x_4'$.

Die Rechte Seite ist dann einfach herzuleiten und das können wir unmittelbar in eine Python Funktion implementieren:

In [ ]:
# Parameters:
t0 = 0;                                 # Startzeit
tE = 17.06521656015796                  # Endzeit - ein voller Orbit
x0 = [0.994, 0, 0, -2.0015851063790825] # Anfangswerte
DEM= 384400                             # Mittlere Entf. Mond-Erde [km] - Einheitslänge
abstol= 1/DEM                           # Toleranz 1 km in Einheitslängen
mu =0.012277471                         # Mondmasseverhältnis (entspricht Position in gewählten Koordinaten)
dt = 0.001                              # Schrittweite für die graphische Darstellung

# Rechte Seite als Funktion
def arent(t,x):
    mup= 1-mu
    x22= x[1]**2
    sqr= x[0]+mu
    r1 = sqr*sqr+ x22
    r1 = r1**(1.5)
    sqr= x[0]- mup
    r2 = sqr*sqr+ x22
    r2 = r2**(1.5)
    ar = [x[2], x[3], x[0]+2*x[3]-mup/r1*(x[0]+mu)-mu/r2*(x[0]-mup), x[1]-2*x[2]-mup/r1*x[1]-mu/r2*x[1]]
    return ar

Als nächstes bestimmen wir die Anzahl der Schritte, an denen wir eine Lösung erwarten, damit wir sie glatt graphisch darstellen können.

In [ ]:
from numpy import floor
n= int(floor(((tE+dt)-t0)/dt))
xx= linspace(t0,tE,n,endpoint=True)

Nun lösen wir das Problem unter Nutzung der SciPy integrate Umgebung und der daraus importierten Funktion `solve_ivp` (das steht für die Lösung von Anfangswertproblemen - initial value problems).

In [ ]:
from scipy.integrate import solve_ivp
sol = solve_ivp(arent,[t0,tE],x0,dense_output=True,rtol=abstol)

s= sol.sol(xx) # Lösung auf dem dichten Gitter zur Visualisierung
xa= s[0,:]     # Extrahiere x-Koord. der Position (x_1)
ya= s[1,:]     # Extrahiere y-Koord. der Position (x_2)

plt.plot(xa,ya,'-g')   # Satellitenbahn
plt.plot(-mu,0,'or')   # Mond
plt.plot(1-mu,0,'ob')  # Erde
plt.axis([-1.5, 1.25, -1.25, 1.25])
plt.title('Arenstorf Orbit')
plt.xlabel('x_1')
plt.ylabel('x_2');

Diesen Orbit hätten wir mit symbolischer Lösung der gewöhnlichen Differentialgleichung wohl nicht so einfach lösen können. Dazu ist die Rechte Seite zu kompliziert.

---
1) <span id="fn1">Copyright Notice:
    
    NumPy Anwendungen, Copyright (C) 2020  Jörn Behrens
    
        Prof. Dr. Jörn Behrens
        Universität Hamburg, Dept. Mathematik
        Bundesstrasse 55
        20146 Hamburg, Germany
        joern.behrens@uni-hamburg.de

    This program is free software; you can redistribute it and/or modify
    it under the terms of the GNU General Public License as published by
    the Free Software Foundation; either version 2 of the License, or
    (at your option) any later version.

    This program is distributed in the hope that it will be useful,
    but WITHOUT ANY WARRANTY; without even the implied warranty of
    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
    GNU General Public License for more details.

    You should have received a copy of the GNU General Public License along
    with this program; if not, write to the Free Software Foundation, Inc.,
    51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.


</span>